# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
#%pip install statsmodels
#%pip install -U scikit-learn

In [2]:
import numpy as np
import pandas as pd
import pylab as plt
from scipy.stats import norm
from scipy.stats import t
from scipy.stats import ttest_1samp 
from scipy.stats import shapiro
from scipy.stats import ttest_rel 
from scipy.stats import ttest_ind  
from scipy.stats import sem 

from statsmodels.stats import weightstats as w 
from statsmodels.stats.proportion import proportions_ztest # test de proporciones

from sklearn.preprocessing import StandardScaler

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents

In [3]:
salaries = pd.read_csv('../Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examine the `salaries` dataset using the `head` function below.

In [4]:
salaries.head()


,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


In [5]:
salaries['Typical Hours'].unique()

array([nan, 20., 40., 35., 10.])

In [6]:
salaries['Hourly Rate'].unique()

array([   nan,  19.86,  46.1 ,  35.6 ,   2.65,  17.68,  21.43,  25.1 ,
        20.  ,  36.21,  45.07,  44.88,  48.25,  36.22,  40.2 ,  36.13,
        32.04,  28.48,  47.5 ,  32.25,  46.35,  21.2 ,  20.77,  13.15,
        21.98,  15.94,  20.12,  12.49,  34.57,  16.17,  13.94,   8.25,
        47.44,  32.59,  43.03,  15.22,  28.14,  19.38,  18.52,  44.25,
        21.3 ,  49.1 ,  15.65,  15.  ,  18.15,  22.36,  35.57,  14.54,
        22.12,  50.25,  38.33,  13.  ,  37.25,  20.31,  40.7 ,  52.18,
        41.1 ,  44.55,  19.5 ,  37.56,  22.35,  41.7 ,  14.51,  37.45,
        12.  ,  53.9 ,  48.9 ,  45.35,  19.61,  22.88,  30.2 ,  24.63,
        18.22,  19.  ,  16.88,  17.42,  45.43,  22.  ,  34.33,  52.1 ,
        23.31,  33.56,  47.8 ,  22.83,   9.27,  14.65,  45.75,  32.16,
        21.55,  16.  ,  24.12,  26.32,  14.23,  10.  ,  14.15,  46.47,
        46.2 ,  48.85,  41.  ,  41.3 ,  46.25,  22.75,  20.24,  20.88,
        30.21,  17.55,  41.72,  50.5 ,  24.41,  10.5 ,  32.09,   9.46,
      

We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [7]:
salaries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33183 entries, 0 to 33182
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               33183 non-null  object 
 1   Job Titles         33183 non-null  object 
 2   Department         33183 non-null  object 
 3   Full or Part-Time  33183 non-null  object 
 4   Salary or Hourly   33183 non-null  object 
 5   Typical Hours      8022 non-null   float64
 6   Annual Salary      25161 non-null  float64
 7   Hourly Rate        8022 non-null   float64
dtypes: float64(3), object(5)
memory usage: 2.0+ MB


In [8]:
salaries.isnull().sum()

Name                     0
Job Titles               0
Department               0
Full or Part-Time        0
Salary or Hourly         0
Typical Hours        25161
Annual Salary         8022
Hourly Rate          25161
dtype: int64

Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [9]:
salaries['Salary or Hourly'].count()

33183

What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [10]:
salaries.groupby('Department')['Name'].count()

Department
ADMIN HEARNG                39
ANIMAL CONTRL               81
AVIATION                  1629
BOARD OF ELECTION          107
BOARD OF ETHICS              8
BUDGET & MGMT               46
BUILDINGS                  269
BUSINESS AFFAIRS           171
CITY CLERK                  84
CITY COUNCIL               411
COMMUNITY DEVELOPMENT      207
COPA                       116
CULTURAL AFFAIRS            65
DISABILITIES                28
DoIT                        99
FAMILY & SUPPORT           615
FINANCE                    560
FIRE                      4641
GENERAL SERVICES           980
HEALTH                     488
HUMAN RELATIONS             16
HUMAN RESOURCES             79
INSPECTOR GEN               87
LAW                        407
LICENSE APPL COMM            1
MAYOR'S OFFICE              85
OEMC                      2102
POLICE                   13414
POLICE BOARD                 2
PROCUREMENT                 92
PUBLIC LIBRARY            1015
STREETS & SAN             21

In [11]:
hourly_stats = salaries[salaries['Salary or Hourly'] == 'Hourly']
hourly_stats.describe()

,Typical Hours,Annual Salary,Hourly Rate
count,8022.000000,0.0,8022.000000
mean,34.507604,NaN,32.788558
std,9.252077,NaN,12.112573
min,10.000000,NaN,2.650000
25%,20.000000,NaN,21.200000
50%,40.000000,NaN,35.600000
75%,40.000000,NaN,40.200000
max,40.000000,NaN,109.000000


# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [12]:
# datos del enunciado
# h0 = 30
# h1 != 30 

mu = 30        # media poblacional

mu_hat = 32.78855771628023    # media muestral, media del experimento

std_hat = 12.112573   # std muestral, std del experimento

n = salaries['Hourly Rate'].dropna()         # tamaño de la muestra

In [13]:
# esto es mu
hourly_wage2 = salaries['Hourly Rate'].mean()
hourly_wage2

32.78855771628023

In [14]:
# obtendremos una muestra, no tenemos toda la poblacion
# muestra = n
n = salaries['Hourly Rate'].dropna()
n

4        19.86
6        46.10
7        35.60
10        2.65
18       17.68
         ...  
33164    46.10
33168    17.68
33169    35.60
33174    46.35
33175    48.85
Name: Hourly Rate, Length: 8022, dtype: float64

In [15]:
t_test = ttest_1samp(salaries['Hourly Rate'].dropna(), mu, alternative='two-sided')

t_test

TtestResult(statistic=20.6198057854942, pvalue=4.3230240486229894e-92, df=8021)

----

In [16]:
t_test.statistic

20.6198057854942

In [17]:
t_test.pvalue
#e-92 bastante igual a 0
# < 0.05
#significa que la hipótesis 0 se puede descartar!!! Es decir, es bastante menor a 0.05. Es significativamente distinto.
# **Podemos rechazar la hipótesis nula**

4.3230240486229894e-92

We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [18]:
#h0 <= $86000/year
#h1 > $86000/year
#greater

mu = 86000        # media poblacional

n = salaries['Annual Salary'].dropna()         # tamaño de la muestra

In [19]:
poblacion = salaries[salaries['Department'] == 'POLICE']
poblacion

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
9,"ABBATE, TERRY M",POLICE OFFICER,POLICE,F,Salary,NaN,93354.0,NaN
11,"ABDALLAH, ZAID",POLICE OFFICER,POLICE,F,Salary,NaN,84054.0,NaN
12,"ABDELHADI, ABDALMAHD",POLICE OFFICER,POLICE,F,Salary,NaN,87006.0,NaN
...,...,...,...,...,...,...,...,...
33177,"ZYGMUNT, DAWID",POLICE OFFICER,POLICE,F,Salary,NaN,72510.0,NaN
33178,"ZYLINSKA, KATARZYNA",POLICE OFFICER,POLICE,F,Salary,NaN,72510.0,NaN
33179,"ZYMANTAS, LAURA C",POLICE OFFICER,POLICE,F,Salary,NaN,48078.0,NaN
33180,"ZYMANTAS, MARK E",POLICE OFFICER,POLICE,F,Salary,NaN,90024.0,NaN


In [20]:
poblacion.describe()

,Typical Hours,Annual Salary,Hourly Rate
count,10.0,13404.000000,10.000000
mean,20.0,86486.414503,9.384000
std,0.0,18272.228294,0.098116
min,20.0,38376.000000,9.270000
25%,20.0,84054.000000,9.270000
50%,20.0,90024.000000,9.460000
75%,20.0,94524.000000,9.460000
max,20.0,260004.000000,9.460000


In [21]:
t_test = ttest_1samp(poblacion['Annual Salary'].dropna(), mu, alternative='greater')

t_test

TtestResult(statistic=3.081997005712994, pvalue=0.0010301701775482569, df=13403)

In [22]:
t_test.pvalue

# el valor es cercano a 0 por lo que podemos rechazar la hipotesis nula. Es decir, es bastante menor a 0.05.

0.0010301701775482569

Using the `crosstab` function, find the department that has the most hourly workers. 

In [28]:
cross = pd.crosstab(salaries[salaries['Salary or Hourly'] == 'Hourly']['Department'] , salaries[salaries['Salary or Hourly'] == 'Hourly']['Name'].count())
cross 
# este es el departamento STREETS & SAN	1862

col_0,8022
Department,
ANIMAL CONTRL,19
AVIATION,1082
BUDGET & MGMT,2
BUSINESS AFFAIRS,7
CITY COUNCIL,64
COMMUNITY DEVELOPMENT,4
CULTURAL AFFAIRS,7
FAMILY & SUPPORT,287
FINANCE,44


The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [25]:
poblacion_dept = salaries[salaries['Department'] == 'STREETS & SAN']
poblacion_dept

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
7,"ABBATE, JOSEPH L",POOL MOTOR TRUCK DRIVER,STREETS & SAN,F,Hourly,40.0,NaN,35.60
21,"ABDUL-SHAKUR, TAHIR",GENERAL LABORER - DSS,STREETS & SAN,F,Hourly,40.0,NaN,21.43
24,"ABERCROMBIE, TIMOTHY",MOTOR TRUCK DRIVER,STREETS & SAN,F,Hourly,40.0,NaN,35.60
36,"ABRAMS, DANIELLE T",SANITATION LABORER,STREETS & SAN,F,Hourly,40.0,NaN,36.21
39,"ABRAMS, SAMUEL A",POOL MOTOR TRUCK DRIVER,STREETS & SAN,F,Hourly,40.0,NaN,35.60
...,...,...,...,...,...,...,...,...
33107,"ZIZUMBO, LUIS",MOTOR TRUCK DRIVER,STREETS & SAN,F,Hourly,40.0,NaN,35.60
33147,"ZUMMO, ROBERT J",MOTOR TRUCK DRIVER,STREETS & SAN,F,Hourly,40.0,NaN,35.60
33149,"ZUNICH, JONATHAN G",SANITATION LABORER,STREETS & SAN,F,Hourly,40.0,NaN,36.21
33156,"ZUNIGA, THOMAS",GENERAL LABORER - DSS,STREETS & SAN,F,Hourly,40.0,NaN,22.12


In [26]:
#h0 = status quo
#h0 >= $35/hour
#h1 < $35/hour
#less

#una queja es siempre h1

mu =35        # media poblacional
n = poblacion_dept['Hourly Rate'].dropna()        # tamaño de la muestra

In [ ]:
len(poblacion_dept['Hourly Rate'].dropna())

1862

In [ ]:
poblacion_dept.describe()

,Typical Hours,Annual Salary,Hourly Rate
count,1862.000000,336.000000,1862.000000
mean,37.137487,83019.250000,33.728378
std,8.777897,21254.991182,5.735242
min,10.000000,31872.000000,19.500000
25%,40.000000,70056.000000,35.600000
50%,40.000000,84168.000000,35.600000
75%,40.000000,97740.000000,36.210000
max,40.000000,154428.000000,53.100000


In [ ]:
t_test = ttest_1samp(poblacion_dept['Hourly Rate'].dropna(), mu, alternative='less')

t_test

TtestResult(statistic=-9.567447887848152, pvalue=1.6689265282353859e-21, df=1861)

In [ ]:
t_test.pvalue
# el valor es cercano a 0 por lo que podemos rechazar la hipotesis nula. Es decir, es bastante menor a 0.05.

1.6689265282353859e-21

# Challenge 3 - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [ ]:
hourly = salaries[salaries["Salary or Hourly"] == "Hourly"]

In [ ]:
hourly.describe()

,Typical Hours,Annual Salary,Hourly Rate
count,8022.000000,0.0,8022.000000
mean,34.507604,NaN,32.788558
std,9.252077,NaN,12.112573
min,10.000000,NaN,2.650000
25%,20.000000,NaN,21.200000
50%,40.000000,NaN,35.600000
75%,40.000000,NaN,40.200000
max,40.000000,NaN,109.000000


In [ ]:
t.interval(0.95,                  # alfa = 1 - pvalor, confianza
           len(hourly["Hourly Rate"])-1,      # N-1, dof
           loc=hourly["Hourly Rate"].mean(),
           scale=sem(hourly["Hourly Rate"])
          )

# este valor tiene q estar entre los 2 resultados q me dan a continuacion 32.788558, 
# como es así hay un 95% de que la media de la muestra se encuentre en estos rangos

(32.52345834488425, 33.05365708767623)

Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [ ]:
police = salaries[salaries['Department'] == 'POLICE']
police

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
9,"ABBATE, TERRY M",POLICE OFFICER,POLICE,F,Salary,NaN,93354.0,NaN
11,"ABDALLAH, ZAID",POLICE OFFICER,POLICE,F,Salary,NaN,84054.0,NaN
12,"ABDELHADI, ABDALMAHD",POLICE OFFICER,POLICE,F,Salary,NaN,87006.0,NaN
...,...,...,...,...,...,...,...,...
33177,"ZYGMUNT, DAWID",POLICE OFFICER,POLICE,F,Salary,NaN,72510.0,NaN
33178,"ZYLINSKA, KATARZYNA",POLICE OFFICER,POLICE,F,Salary,NaN,72510.0,NaN
33179,"ZYMANTAS, LAURA C",POLICE OFFICER,POLICE,F,Salary,NaN,48078.0,NaN
33180,"ZYMANTAS, MARK E",POLICE OFFICER,POLICE,F,Salary,NaN,90024.0,NaN


In [ ]:
pol_clean = police["Annual Salary"].dropna()
pol_clean

0        101442.0
1         94122.0
9         93354.0
11        84054.0
12        87006.0
           ...   
33177     72510.0
33178     72510.0
33179     48078.0
33180     90024.0
33181     93354.0
Name: Annual Salary, Length: 13404, dtype: float64

In [ ]:
pol_clean.describe()

count     13404.000000
mean      86486.414503
std       18272.228294
min       38376.000000
25%       84054.000000
50%       90024.000000
75%       94524.000000
max      260004.000000
Name: Annual Salary, dtype: float64

In [ ]:
t.interval(0.95,                  
           len(pol_clean)-1,     
           loc=pol_clean.mean(),
           scale=sem(pol_clean)
          )

# este valor tiene q estar entre los 2 resultados q me dan a continuacion 32.788558, 
# como es así hay un 95% de que la media de la muestra se encuentre en estos rangos


(86177.05631531784, 86795.77269094894)

# Challenge 4 - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [ ]:
#h0 = 25%
#h1 != 25%

#si no sabemos si esta x encima o x por abajo, es el de 2 colas!!

In [29]:
salaries['Salary or Hourly'].value_counts()

Salary or Hourly
Salary    25161
Hourly     8022
Name: count, dtype: int64

In [30]:
salaries.shape[0]

33183

In [31]:
33183/4

8295.75

In [ ]:
z_score, p_value= proportions_ztest(salaries['Salary or Hourly'].value_counts()['Hourly'],  #elementos  
                                    len(salaries['Salary or Hourly']),  # numero total de elementos
                                    0.25            # proporcion
                                   ) 

z_score, p_value

(-3.5099964213703005, 0.0004481127249057967)

In [ ]:
p_value

# el valor es cercano a 0 por lo que podemos rechazar la hipotesis nula. Es decir, es bastante menor a 0.05.
# hay evidencia significativa para rechazar la hipótesis nula.
# afirmamos que es significativamente distinta del 25%, pero no cuan diferente en magnitud

0.0004481127249057967